In [8]:
import pandas as pd
import torch
from kan import KAN
from transformers import AutoModel, AutoTokenizer
from transformers import BertModel, BertTokenizer
import torch.nn as nn
torch.set_default_dtype(torch.float32)


In [9]:
df = pd.read_csv('/tmp/pycharm_project_763/data/trainCPP.csv')

# 提取序列和标签
sequences = df['sequence'].tolist()
y = df['label'].tolist()

df2= pd.read_csv('/tmp/pycharm_project_763/data/testCPP.csv')

# 提取序列和标签
sequences_test = df2['sequence'].tolist()
y_test = df2['label'].tolist()

In [10]:
def Tokenizer(X,y) :
    device = torch.device("cuda:1")
    model = AutoModel.from_pretrained("facebook/esm2_t12_35M_UR50D")
    model.to(device)
    tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t12_35M_UR50D")
    tokenizer_pro = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
    model_pro = BertModel.from_pretrained("Rostlab/prot_bert")
    # model_pro.to(device)
    dropout = nn.Dropout(0.2)
    fc_pro = nn.Linear(1024, 480)

    inputs = tokenizer(X, padding=True, truncation=True, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    encoded_input = tokenizer_pro(X, padding=True, truncation=True,return_tensors='pt')
    outputs_pro = model_pro(**encoded_input)
    pooler_output1 = outputs.pooler_output
    pooler_output2=outputs_pro.pooler_output
    pooler_output2=fc_pro(pooler_output2)
    pooler_output2=pooler_output2.to(device)
    X =pooler_output1
    return X

In [11]:


X_train,y_train=Tokenizer(sequences,y)
X_test,y_test=Tokenizer(sequences_test,y_test)

Some weights of the model checkpoint at facebook/esm2_t12_35M_UR50D were not used when initializing EsmModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['esm.pooler.dense.weight', 'esm.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model 

RuntimeError: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility

In [ ]:

X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

In [ ]:
y_train = y_train.reshape((y_train.shape[0], 1))
y_test = y_test.reshape((y_test.shape[0], 1))

dataset={}
dataset = {
        'train_input': X_train,
        'test_input': X_test,
        'train_label': y_train,
        'test_label': y_test
    }

In [ ]:
model = KAN(width=[480,5,1], grid=3, k=2)

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef
def train_acc():
    pred=model(X_train)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_train.cpu(), pred.cpu())
    
    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_train.cpu(), pred_labels.cpu()).ravel()
        
        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_train.cpu(), pred_labels.cpu())
    print(f"AUC: {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_train)[:, 0]) == y_train[:, 0]).float())

def test_acc():
    return torch.mean((torch.round(model(X_test)[:, 0]) == y_test[:, 0]).float())

# results = model.train(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc))
results = model.fit(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc));
print(results['train_acc'][-1], results['test_acc'][-1])